# Rerun.io — LSD Visualization Demo

This notebook demonstrates how to visualize line segment detection results
using [Rerun](https://rerun.io) directly inside Jupyter.

**Prerequisites:**
- Build bindings: `bazel build //libs/...`
- Install rerun with notebook support: `pip install "rerun-sdk[notebook]"`
- The `.venv` should already have it if you ran `uv sync` after updating `pyproject.toml`.

---
## 1. Setup & Imports

In [ ]:
import sys, pathlib

# --- Locate workspace root and add Bazel output dirs to sys.path ---
workspace = pathlib.Path.cwd()
while not (workspace / "MODULE.bazel").exists():
    if workspace == workspace.parent:
        raise RuntimeError("Cannot find LineExtraction workspace root (MODULE.bazel)")
    workspace = workspace.parent

# Add each binding's Bazel output directory
for lib in ["imgproc", "edge", "geometry", "eval", "lsd"]:
    p = workspace / f"bazel-bin/libs/{lib}/python"
    if p.exists():
        sys.path.insert(0, str(p))
    else:
        print(f"Warning: Not found: {p}  — run: bazel build //libs/{lib}/...")

# Add lsfm package
sys.path.insert(0, str(workspace / "python"))

import colorsys
import numpy as np
import rerun as rr
import le_lsd
from lsfm.data import TestImages

print(f"Workspace: {workspace}")
print(f"rerun version: {rr.__version__}")
print(f"le_lsd loaded: {len([x for x in dir(le_lsd) if not x.startswith('_')])} symbols")

---
## 2. Initialize Rerun for notebook display

We use `rr.init()` and then `rr.notebook_show()` later to render the Rerun
viewer inline as a Jupyter widget.

In [ ]:
rr.init("rerun_lsd_notebook")
print("Rerun initialized.")

---
## 3. Load test image

In [ ]:
from PIL import Image

test_images = TestImages()
img_path = str(test_images.windmill())
img_u8 = np.array(Image.open(img_path).convert("L"), dtype=np.uint8)

print(f"Image: {img_path}")
print(f"Shape: {img_u8.shape}")

# Log the input image
rr.log("image/original", rr.Image(img_u8))

---
## 4. Run LSD detectors & log results

We run several detectors and log each as a separate timeline step so you can
scrub through them in the Rerun Viewer.

Each line segment is logged with a **per-instance label** containing its index,
length, and angle. **Click any segment** in the 2D view to see its properties
in the Selection Panel on the right.

In [ ]:
import math


def generate_colors(n: int) -> np.ndarray:
    """Generate n visually distinct RGB colors via HSV spacing."""
    colors = []
    for i in range(max(n, 1)):
        hue = i / max(n, 1)
        r, g, b = colorsys.hsv_to_rgb(hue, 0.9, 0.9)
        colors.append([int(r * 255), int(g * 255), int(b * 255)])
    return np.array(colors, dtype=np.uint8)


DETECTORS = {
    "CC": "LsdCC",
    "Burns": "LsdBurns",
    "FGioi": "LsdFGioi",
    "EDLines": "LsdEDLZ",
    "EL": "LsdEL",
}

for step, (name, cls_name) in enumerate(DETECTORS.items()):
    rr.set_time("detector_step", sequence=step)

    det = getattr(le_lsd, cls_name)()
    det.detect(img_u8)
    segments = det.line_segments()

    # Log input image at each step (so it is visible at every timestep)
    rr.log("image/original", rr.Image(img_u8))

    # Build line strips, endpoints, and per-segment labels
    strips = []
    points = []
    seg_labels = []
    ep_labels = []

    for i, seg in enumerate(segments):
        ep = seg.end_points()  # (x1, y1, x2, y2)
        x1, y1, x2, y2 = float(ep[0]), float(ep[1]), float(ep[2]), float(ep[3])
        strips.append([[x1, y1], [x2, y2]])
        points.append([x1, y1])
        points.append([x2, y2])

        # Per-segment label with index, length, and angle
        angle_deg = math.degrees(seg.angle)
        seg_labels.append(
            f"#{i}  L={seg.length:.1f}  θ={angle_deg:.1f}°"
        )
        ep_labels.append(f"#{i} start ({x1:.0f}, {y1:.0f})")
        ep_labels.append(f"#{i} end ({x2:.0f}, {y2:.0f})")

    seg_colors = generate_colors(len(segments))

    # Log all segments as a single batch — each strip is a selectable instance
    # with its own color and label (visible on hover / click in the viewer).
    rr.log(
        "image/segments",
        rr.LineStrips2D(
            strips,
            colors=seg_colors,
            labels=seg_labels,
            show_labels=False,  # labels visible in Selection Panel, not cluttering the view
            radii=1.5,
        ),
    )

    # Log endpoints with labels identifying which segment they belong to
    rr.log(
        "image/endpoints",
        rr.Points2D(
            points,
            colors=[255, 60, 60],
            labels=ep_labels,
            show_labels=False,
            radii=2.5,
        ),
    )

    rr.log("logs", rr.TextLog(f"[{name}] {len(segments)} segments detected"))

    # Log auxiliary image data layers
    try:
        desc = det.image_data_descriptor()
        data = det.image_data()
        for i in range(min(len(desc), len(data))):
            layer = data[i].astype(np.float64)
            if layer.max() > layer.min():
                layer_norm = ((layer - layer.min()) / (layer.max() - layer.min()) * 255).astype(np.uint8)
            else:
                layer_norm = np.zeros_like(data[i], dtype=np.uint8)
            rr.log(f"image/data/{desc[i].name}", rr.Image(layer_norm))
    except Exception:
        pass

    print(f"  {name:10s} ({cls_name:12s}): {len(segments):4d} segments")

print("\nDone. All detectors logged.")

---
## 5. Show Rerun Viewer inline

The cell below renders the Rerun Viewer directly in the notebook.

**Navigating the data:**
- **Timeline scrubber** (bottom) — switch between detectors
- **Click a line segment** — the Selection Panel (right) shows its label with index, length, and angle
- **Hover** over segments or endpoints to see coordinates and labels
- **Entity tree** (left) — toggle `image/segments`, `image/endpoints`, `image/data/*` visibility

> **Note:** This requires `rerun-notebook` (installed via `pip install "rerun-sdk[notebook]"`).
> The widget loads a ~31 MB Wasm viewer — first load may take a few seconds.

In [ ]:
rr.notebook_show(width=1600, height=1080)

---
## 6. Alternative: spawn external viewer

If you prefer the full desktop Rerun Viewer instead of the inline widget,
uncomment and run the cell below. The viewer will open in a separate window.

In [ ]:
# Uncomment to use the external viewer instead of inline widget:
# rr.init("rerun_lsd_notebook", spawn=True)
# ... then re-run the detection cells above.